In [1]:
# import wymaganych bibliotek
import requests
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
import json
from datetime import datetime
import os
import xlsxwriter
import warnings
# wyłączenie ostrzeżeń związanych z wieloktronym zamykaniem pliku excel
warnings.filterwarnings("ignore")
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

#### Functions

In [2]:
# funkcja otwiera i zwraca plik xlsx/xls lub csv z wprowadzonej ścieżki
def read_csv_xlsx(path):
    try:
        if '.csv' in path:
            file = pd.read_csv(path, names = ['Waluty'], header = None)
            print(f'\nPlik csv załadowany')
        elif '.xls' in path:
            file = pd.read_excel(path, names = ['Waluty'], header = None)
            print(f'\nPlik xlsx załadowany')
# funkcja wyświetla błąd w przypadku nie znalezienia pliku w danej ścieżce
        else:
            print(f'\nNiepoprawna ścieżka - nie znaleziono pliku - '+ path.rsplit('\\',1)[1])
    except ValueError:
# funkcja zwraca błąd jeżeli plik wsadowy zawiera więcej niż jednąć kolumnę
        print(f'\nW pliku źródłowym jest więcej niż 1 kolumna!')
    else:
        return file

In [18]:
# funkcja wysyła prośbę do api strony NBP i jeżeli otrzyma kod HTTP OK - 200 - zwraca tabelę z kursami walut za ostatnie 30 dni
# funkcja wyświetla błąd w przypadku nie znalezienia waluty - jednoznaczne z otrzymaniem kodu HTTP innego niż 200
def nbp_api_request(requested_currency_abv):
    currency_table = requests.get(f'http://api.nbp.pl/api/exchangerates/rates/A/{requested_currency_abv}/last/30/')
    if currency_table.status_code == 404:
        print('Błąd połączenia - nie znaleziono waluty ' + str(requested_currency_abv))
    elif currency_table.status_code != 200:
        print('Błąd połączenia - error:' + str(currency_table.status_code))
    else:
        return  currency_table

In [4]:
# funkcja zwraca pełną ścieżkę pliku jeżeli uda się sparować wprowadzoną do funkcji nazwę z nazwą jednego z plików w aktualnej ścieżce
def get_path(file):
    for i in os.listdir():
        y = None
        y = i.split('.')[0]
        file = file.split('.')[0]
        if file.lower() == y.lower():
            full_path = curr_dir + i
            return full_path

In [5]:
# def jprint(obj):
#     # create a formatted string of the Python JSON object
#     text = json.dumps(obj, sort_keys=True, indent=4)
#     print(text)

### Workbench:

In [ ]:
#prośba o wprowadzenie kodu waluty lub wybranie pliku z listą walut
user_response = input('Wpisz jeden kod waluty np. EUR, CHF, DKK lub wpisz CSV dla listy kodów zawartej w pliku .csv/.xls/.xlsx: ')
# określenie aktualnej ścieżki
curr_dir = os.getcwd() + '\\'
#jeżeli użytkownik wprowadził jeden kod walut:
if user_response.lower() != 'csv':
    #wprowadzony symbol zamieniany jest na dataFrame
    currencies = pd.DataFrame([user_response.upper()],columns=['Waluty'])
#jeżeli użytkownik wprowadził określenie CSV w celu podania pliku z listą walut:
else:
    
    print(f'\n')
    # wylistowanie wszystkich dostępnych plików w aktualnej ścieżce z rozszerzeniem zawierającym zwroty xls/csv
    for i in os.listdir():
        if any(x in i for x in ['csv','xls']):
            print(i)
    full_path = None
    # pętla działająca dopóki nie zostanie wprowadzona nazwa pliku, którą można sparować z jednym z plików w aktualnej ścieżce
    # powinna zostać wprowadzona nazwa jednego z plików wylistowanych w pętli for powyżej
    while full_path == None:
        full_path = None
        plik = input(f'\nWybierz plik z powyższej listy skoroszytów znajdujących się w aktualnym folderze. \nPlik powinien zawierać tylko kolumnę z listą wybranych kodów!\n')
        full_path = get_path(plik)
        if full_path == None: print(f'\nNie znaleziono "'+ plik +'" w folderze.')
    # po znalezieniu odpowiedniego pliku - plik ten zostaje zczytany przez funkcję read_csv_xlsx - biblioteka pandas
    currencies = read_csv_xlsx(full_path)
# ustalenie jakiego typu zapytanie zostało utworzone - dla 1 waluty czy dla wielu
choice_type = len(currencies)
# XLSX FILE SET UP
# zależnie od długości pandas data frame ustalana jest nazwa dla pliku z jedną walutą:
if choice_type == 1:
    wb_name = 'Tabela_'+ str(currencies['Waluty'][0])+ '_' + datetime.today().strftime(('%Y_%m_%d'))
# lub wieloma walutami:
else:
    wb_name = 'Tabele_walut_' + datetime.today().strftime(('%Y_%m_%d'))
# ustalenie ścieżkid dla pliku docelowego:
path_wb = os.path.join(curr_dir,wb_name + '.xlsx')
#workbook = xlsxwriter.Workbook(path_wb)
# definicja silnika zapisu w excelu poprzez bibliotekę pandas
writer_engine = pd.ExcelWriter(path_wb, date_format = 'YYYY-MM-DD', datetime_format='YYYY-MM-DD', engine='xlsxwriter')
# utworzenie w silniku obiektu skoroszytu
workbook = writer_engine.book
#try:
# pętla wykonująca czynności dla kolejnych walut zdefiniowanych wcześniej w data frame - z listą walut wybranych przez użytkownika
for currency in currencies['Waluty']: 
### API REQUEST - zapytanie API NBP o daną walutę przy pomocy wcześniej zdefiniowanej funkcji nbp_api_request
    nbp_request = nbp_api_request(currency)
    #print(nbp_request)
    if nbp_request == None: continue
### DATAFRAME -  tworzenie data frame na podstawie otrzymanych wyników z API. 
#                Poniżej zdefiniowana tabela z kursami zostanie umieszczona w excelu.
    df = pd.DataFrame(nbp_request.json()['rates'])
    df.rename(columns = {'no':'Tabela_NBP','effectiveDate':'Data_publikacji','mid':'Kurs_średni'}, inplace = True)
    df.Data_publikacji = pd.to_datetime(df.Data_publikacji,format = '%Y-%m-%d')
    df.reset_index(drop = True, inplace = True)

    # obliczanie przyrostów względnych, bezwzględnych, średniego przyrostu z ostatnich 30 dni oraz 7-dniowej średniej ruchomej
    df['Przyrost_bezwzględny'] = df['Kurs_średni'].diff()
    df['Przyrost_względny'] = df['Kurs_średni'].pct_change()
    df['Przyrost_względny'] = df['Przyrost_względny'].round(4)
    df['Średnia_ruchoma'] = df['Kurs_średni'].rolling(window = 7).mean()
    df['Średnia_ruchoma'] = df['Średnia_ruchoma'].round(5)
    df.sort_values('Data_publikacji', ascending = False,inplace = True)
    mean_change = df['Przyrost_względny'].mean().round(5)
    
    
### WORKSHEET
# ustalenie wielkości tabeli oraz rozproszenia wartości kursów średnich
    max_row = len(df)
    max_kurs = max(df['Kurs_średni'])
    min_kurs = min(df['Kurs_średni'])
#    max_range_graph = round( max_kurs + 0.01,2)
#    min_range_graph = round( min_kurs - 0.01,2)
    sheet = 'Tabela '+ currency
# zapisanie tabeli kursów w aktualnie zdefiniowanym arkuszu     
    df.to_excel(writer_engine,sheet_name = sheet, index = False)
    curr_sheet = writer_engine.sheets[sheet]
# ustawienie szerokości kolumn
    curr_sheet.set_column('A:H',18)
    curr_sheet.set_column('I:I', 26) 
 ### CHART
     #przygotowanie obiektu grafu
    chart = workbook.add_chart({'type': 'line'})
     # przygotowanie serii danych dla grafu - kurs średni oraz serii - średnia ruchoma 7 dniowa
    chart.add_series({
            'name':'Kurs ' + currency +'/PLN',
            'categories':[sheet, 1, 1,   max_row, 1],
             'values':[sheet, 1, 2, max_row, 2],
             })
    chart.add_series({
            'name':'Średnia ruchoma 7 dniowa',
            'categories':[sheet, 1, 1,   max_row, 1],
             'values':[sheet, 1, 5, max_row, 5],
             })
    
    chart.set_x_axis({'name': 'Data'})
    
     # formatowanie odstępów siatki osi na podstawie wartości kursu
     # różnica między wartością maksymalną a minimalną kursu w ciągu 30 dni określa
     # dystans pomiędzy liniami siatki na wykresie
    if max_kurs -min_kurs > 0.02:
        max_range_graph = round( max_kurs + 0.01,2)
        min_range_graph = round( min_kurs - 0.01,2)
        chart.set_y_axis({'name': 'Kurs średni '+ currency + '/PLN' ,
                      'min':min_range_graph,'max':max_range_graph,
                      'major_gridlines': {'visible': True}, 'major_unit':0.01})
    elif max_kurs-min_kurs > 0.01:
        max_range_graph = round( max_kurs + 0.002,3)
        min_range_graph = round( min_kurs - 0.002,3)
        chart.set_y_axis({'name': 'Kurs średni '+ currency + '/PLN' ,
                      'min':min_range_graph,'max':max_range_graph,
                      'major_gridlines': {'visible': True}, 'major_unit':0.001})
    else:
        max_range_graph = round( max_kurs + 0.0006,4)
        min_range_graph = round( min_kurs - 0.0006,4)
        chart.set_y_axis({'name': 'Kurs średni '+ currency + '/PLN' ,
                      'min':min_range_graph,'max':max_range_graph,
                      'major_gridlines': {'visible': True}, 'major_unit':0.0005})
    chart.set_legend({'position':'top'})
    
    #wstawianie grafu oraz wartości min, max kursu i średniego przyrostu z ostatnich 30 dni do arkusza
    cell_format = workbook.add_format()
    cell_format.set_bold()
    cell_format.set_align('center')
    curr_sheet.insert_chart('H4', chart, {'x_scale': 2.1, 'y_scale': 2.1} )
    curr_sheet.write_string('G1', 'Maksimum kursów',cell_format)
    curr_sheet.write_string('H1', 'Minimum kursów',cell_format)
    curr_sheet.write_string('I1', 'Średni przyrost w ciągu 30 dni',cell_format)
    curr_sheet.write_number('G2', max_kurs)
    curr_sheet.write_number('H2', min_kurs)
    curr_sheet.write_number('I2', mean_change)
    
    print(currency + ' added')
print('Finished')
# zapisanie i zamknięcie silnika zapisywania w excel w celu odłączenia się od pliku
writer_engine.save()
workbook.close()
writer_engine.close()
writer_engine.handles = None

#except:
 #print('Wystąpił błąd podczas przetwarzania pliku')
